# Setup MediaPipe - Procesamiento de Videos
**Sistema de Anotación de Video - Entrega 1**

Este notebook procesa videos de actividades humanas y extrae coordenadas usando MediaPipe.

## 🎯 Proceso Simple:
1. **Subir videos por actividad** → 2. **Procesar con MediaPipe** → 3. **Generar CSVs**

## Actividades:
- **caminar_hacia** - **caminar_regreso** - **girar** - **sentarse** - **ponerse_pie**

---


## Paso 1: Instalación
Ejecutar esta celda para instalar todas las dependencias necesarias.


In [ ]:

# En Colab o local, instalar MediaPipe y OpenCV
!pip install mediapipe==0.10.21 numpy==1.26.4 protobuf==4.25.8 --upgrade --force-reinstall
# Instalar dependencias
!pip install opencv-python matplotlib pandas tqdm -q

# Imports
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
import os
from pathlib import Path
from tqdm import tqdm
import json
import shutil
from google.colab import files


  Using cached mediapipe-0.10.21-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.7 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached protobuf-4.25.8-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached flatbuffers-25.9.23-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached jax-0.8.0-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.8.0-cp312-cp312-manylinux_2_27_x86_64.whl.metadata (1.3 kB)
  Using cached matplotlib-3.10.7-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached sounddevice-0.5.3-py3-none-any.whl.metadata (1.6 kB)
  Using cached sentencepiece-0.2.1-cp312-cp312-manylinux_2_27_x86_64.manylin

/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


## Paso 2: Configuración Inicial
Crear carpetas necesarias para el proyecto.


In [ ]:
# Configuración y carpetas
ACTIVITIES = ['caminar_hacia', 'caminar_regreso', 'girar', 'sentarse', 'ponerse_pie']

# Crear carpetas
for folder in ['data/videos', 'data/landmarks', 'data/metadata']:
    Path(folder).mkdir(parents=True, exist_ok=True)

for activity in ACTIVITIES:
    Path(f"data/videos/{activity}").mkdir(parents=True, exist_ok=True)

print("✅ Carpetas creadas:")
for activity in ACTIVITIES:
    print(f"   📁 data/videos/{activity}/")
print("   📁 data/landmarks/")
print("   📁 data/metadata/")


✅ Carpetas creadas:
   📁 data/videos/caminar_hacia/
   📁 data/videos/caminar_regreso/
   📁 data/videos/girar/
   📁 data/videos/sentarse/
   📁 data/videos/ponerse_pie/
   📁 data/landmarks/
   📁 data/metadata/


## Paso 3.1: Subir Videos - CAMINAR HACIA LA CÁMARA
Subir todos los videos de personas caminando hacia la cámara.


In [ ]:
from google.colab import files  # Para subir archivos
import shutil                   # Para mover archivos
from pathlib import Path        # Para manejar rutas
# SUBIR VIDEOS: CAMINAR HACIA LA CÁMARA
print("🚶 SUBIR VIDEOS: CAMINAR HACIA LA CÁMARA")
print("=" * 50)

uploaded = files.upload()

count = 0
if uploaded:
    for filename in uploaded.keys():
        # Renombrar con formato estándar
        new_name = f"caminar_hacia_{count+1:03d}.mp4"
        dest_path = Path(f"data/videos/caminar_hacia/{new_name}")
        shutil.move(filename, str(dest_path))
        count += 1
        print(f"✅ {filename} → {new_name}")

print(f"📊 Videos de 'caminar hacia': {count}")


🚶 SUBIR VIDEOS: CAMINAR HACIA LA CÁMARA


Saving 1-1.mp4 to 1-1.mp4
Saving 2-1.mp4 to 2-1.mp4
Saving 3-1.mp4 to 3-1.mp4
Saving 4-1.mp4 to 4-1.mp4
Saving 5-1.mp4 to 5-1.mp4
Saving 6-1.mp4 to 6-1.mp4
Saving 7-1.mp4 to 7-1.mp4
Saving 8-1.mp4 to 8-1.mp4
Saving 9-1.mp4 to 9-1.mp4
Saving 10-1.mp4 to 10-1.mp4
Saving 11-1.mp4 to 11-1.mp4
Saving 12-1.mp4 to 12-1.mp4
Saving 13-1.mp4 to 13-1.mp4
Saving 14-1.mp4 to 14-1.mp4
Saving 15-1.mp4 to 15-1.mp4
Saving 16-1.mp4 to 16-1.mp4
Saving 17-1.mp4 to 17-1.mp4
Saving 18-1.mp4 to 18-1.mp4
✅ 1-1.mp4 → caminar_hacia_001.mp4
✅ 2-1.mp4 → caminar_hacia_002.mp4
✅ 3-1.mp4 → caminar_hacia_003.mp4
✅ 4-1.mp4 → caminar_hacia_004.mp4
✅ 5-1.mp4 → caminar_hacia_005.mp4
✅ 6-1.mp4 → caminar_hacia_006.mp4
✅ 7-1.mp4 → caminar_hacia_007.mp4
✅ 8-1.mp4 → caminar_hacia_008.mp4
✅ 9-1.mp4 → caminar_hacia_009.mp4
✅ 10-1.mp4 → caminar_hacia_010.mp4
✅ 11-1.mp4 → caminar_hacia_011.mp4
✅ 12-1.mp4 → caminar_hacia_012.mp4
✅ 13-1.mp4 → caminar_hacia_013.mp4
✅ 14-1.mp4 → caminar_hacia_014.mp4
✅ 15-1.mp4 → caminar_hacia_015.mp

## Paso 3.2: Subir Videos - CAMINAR DE REGRESO
Subir todos los videos de personas caminando alejándose de la cámara.


In [ ]:
# SUBIR VIDEOS: CAMINAR DE REGRESO
print("🚶 SUBIR VIDEOS: CAMINAR DE REGRESO")
print("=" * 50)

uploaded = files.upload()

count = 0
if uploaded:
    for filename in uploaded.keys():
        new_name = f"caminar_regreso_{count+1:03d}.mp4"
        dest_path = Path(f"data/videos/caminar_regreso/{new_name}")
        shutil.move(filename, str(dest_path))
        count += 1
        print(f"✅ {filename} → {new_name}")

print(f"📊 Videos de 'caminar regreso': {count}")


🚶 SUBIR VIDEOS: CAMINAR DE REGRESO


Saving 1-1.mp4 to 1-1.mp4
Saving 2-1.mp4 to 2-1.mp4
Saving 3-1.mp4 to 3-1.mp4
Saving 4-1.mp4 to 4-1.mp4
Saving 5-1.mp4 to 5-1.mp4
Saving 6-1.mp4 to 6-1.mp4
Saving 7-1.mp4 to 7-1.mp4
Saving 8-1.mp4 to 8-1.mp4
Saving 9-1.mp4 to 9-1.mp4
Saving 10-1.mp4 to 10-1.mp4
Saving 11-1.mp4 to 11-1.mp4
Saving 12-1.mp4 to 12-1.mp4
Saving 13-1.mp4 to 13-1.mp4
Saving 14-1.mp4 to 14-1.mp4
Saving 15-1.mp4 to 15-1.mp4
Saving 16-1.mp4 to 16-1.mp4
Saving 17-1.mp4 to 17-1.mp4
Saving 18-1.mp4 to 18-1.mp4
✅ 1-1.mp4 → caminar_regreso_001.mp4
✅ 2-1.mp4 → caminar_regreso_002.mp4
✅ 3-1.mp4 → caminar_regreso_003.mp4
✅ 4-1.mp4 → caminar_regreso_004.mp4
✅ 5-1.mp4 → caminar_regreso_005.mp4
✅ 6-1.mp4 → caminar_regreso_006.mp4
✅ 7-1.mp4 → caminar_regreso_007.mp4
✅ 8-1.mp4 → caminar_regreso_008.mp4
✅ 9-1.mp4 → caminar_regreso_009.mp4
✅ 10-1.mp4 → caminar_regreso_010.mp4
✅ 11-1.mp4 → caminar_regreso_011.mp4
✅ 12-1.mp4 → caminar_regreso_012.mp4
✅ 13-1.mp4 → caminar_regreso_013.mp4
✅ 14-1.mp4 → caminar_regreso_014.mp4
✅ 15-

## Paso 3.3: Subir Videos - GIRAR
Subir todos los videos de personas girando 360°.


In [ ]:
# SUBIR VIDEOS: GIRAR
print("🔄 SUBIR VIDEOS: GIRAR")
print("=" * 50)

uploaded = files.upload()

count = 0
if uploaded:
    for filename in uploaded.keys():
        new_name = f"girar_{count+1:03d}.mp4"
        dest_path = Path(f"data/videos/girar/{new_name}")
        shutil.move(filename, str(dest_path))
        count += 1
        print(f"✅ {filename} → {new_name}")

print(f"📊 Videos de 'girar': {count}")


🔄 SUBIR VIDEOS: GIRAR


Saving 1-1.mp4 to 1-1.mp4
Saving 2-1.mp4 to 2-1.mp4
Saving 3-1.mp4 to 3-1.mp4
Saving 4-1.mp4 to 4-1.mp4
Saving 5-1.mp4 to 5-1.mp4
Saving 6-1.mp4 to 6-1.mp4
Saving 7-1.mp4 to 7-1.mp4
Saving 8-1.mp4 to 8-1.mp4
Saving 9-1.mp4 to 9-1.mp4
Saving 10-1.mp4 to 10-1.mp4
Saving 11-1.mp4 to 11-1.mp4
Saving 12-1.mp4 to 12-1.mp4
Saving 13-1.mp4 to 13-1.mp4
Saving 14-1.mp4 to 14-1.mp4
Saving 15-1.mp4 to 15-1.mp4
Saving 16-1.mp4 to 16-1.mp4
Saving 17-1.mp4 to 17-1.mp4
Saving 18-1.mp4 to 18-1.mp4
✅ 1-1.mp4 → girar_001.mp4
✅ 2-1.mp4 → girar_002.mp4
✅ 3-1.mp4 → girar_003.mp4
✅ 4-1.mp4 → girar_004.mp4
✅ 5-1.mp4 → girar_005.mp4
✅ 6-1.mp4 → girar_006.mp4
✅ 7-1.mp4 → girar_007.mp4
✅ 8-1.mp4 → girar_008.mp4
✅ 9-1.mp4 → girar_009.mp4
✅ 10-1.mp4 → girar_010.mp4
✅ 11-1.mp4 → girar_011.mp4
✅ 12-1.mp4 → girar_012.mp4
✅ 13-1.mp4 → girar_013.mp4
✅ 14-1.mp4 → girar_014.mp4
✅ 15-1.mp4 → girar_015.mp4
✅ 16-1.mp4 → girar_016.mp4
✅ 17-1.mp4 → girar_017.mp4
✅ 18-1.mp4 → girar_018.mp4
📊 Videos de 'girar': 18


## Paso 3.4: Subir Videos - SENTARSE
Subir todos los videos de personas sentándose.


In [ ]:
# SUBIR VIDEOS: SENTARSE
print("💺 SUBIR VIDEOS: SENTARSE")
print("=" * 50)

uploaded = files.upload()

count = 0
if uploaded:
    for filename in uploaded.keys():
        new_name = f"sentarse_{count+1:03d}.mp4"
        dest_path = Path(f"data/videos/sentarse/{new_name}")
        shutil.move(filename, str(dest_path))
        count += 1
        print(f"✅ {filename} → {new_name}")

print(f"📊 Videos de 'sentarse': {count}")


💺 SUBIR VIDEOS: SENTARSE


Saving 1-1.mp4 to 1-1.mp4
Saving 2-1.mp4 to 2-1.mp4
Saving 3-1.mp4 to 3-1.mp4
Saving 4-1.mp4 to 4-1.mp4
Saving 5-1.mp4 to 5-1.mp4
Saving 6-1.mp4 to 6-1.mp4
Saving 7-1.mp4 to 7-1.mp4
Saving 8-1.mp4 to 8-1.mp4
Saving 9-1.mp4 to 9-1.mp4
Saving 10-1.mp4 to 10-1.mp4
Saving 11-1.mp4 to 11-1.mp4
Saving 12-1.mp4 to 12-1.mp4
Saving 13-1.mp4 to 13-1.mp4
Saving 14-1.mp4 to 14-1.mp4
Saving 15-1.mp4 to 15-1.mp4
Saving 16-1.mp4 to 16-1.mp4
Saving 17-1.mp4 to 17-1.mp4
Saving 18-1.mp4 to 18-1.mp4
✅ 1-1.mp4 → sentarse_001.mp4
✅ 2-1.mp4 → sentarse_002.mp4
✅ 3-1.mp4 → sentarse_003.mp4
✅ 4-1.mp4 → sentarse_004.mp4
✅ 5-1.mp4 → sentarse_005.mp4
✅ 6-1.mp4 → sentarse_006.mp4
✅ 7-1.mp4 → sentarse_007.mp4
✅ 8-1.mp4 → sentarse_008.mp4
✅ 9-1.mp4 → sentarse_009.mp4
✅ 10-1.mp4 → sentarse_010.mp4
✅ 11-1.mp4 → sentarse_011.mp4
✅ 12-1.mp4 → sentarse_012.mp4
✅ 13-1.mp4 → sentarse_013.mp4
✅ 14-1.mp4 → sentarse_014.mp4
✅ 15-1.mp4 → sentarse_015.mp4
✅ 16-1.mp4 → sentarse_016.mp4
✅ 17-1.mp4 → sentarse_017.mp4
✅ 18-1.mp4 → 

## Paso 3.5: Subir Videos - PONERSE DE PIE
Subir todos los videos de personas levantándose.


In [ ]:
# SUBIR VIDEOS: PONERSE DE PIE
print("🚶 SUBIR VIDEOS: PONERSE DE PIE")
print("=" * 50)

uploaded = files.upload()

count = 0
if uploaded:
    for filename in uploaded.keys():
        new_name = f"ponerse_pie_{count+1:03d}.mp4"
        dest_path = Path(f"data/videos/ponerse_pie/{new_name}")
        shutil.move(filename, str(dest_path))
        count += 1
        print(f"✅ {filename} → {new_name}")

print(f"📊 Videos de 'ponerse de pie': {count}")


🚶 SUBIR VIDEOS: PONERSE DE PIE


Saving 1-1.mp4 to 1-1.mp4
Saving 2-1.mp4 to 2-1.mp4
Saving 3-1.mp4 to 3-1.mp4
Saving 4-1.mp4 to 4-1.mp4
Saving 5-1.mp4 to 5-1.mp4
Saving 6-1.mp4 to 6-1.mp4
Saving 7-1.mp4 to 7-1.mp4
Saving 8-1.mp4 to 8-1.mp4
Saving 9-1(1).mp4 to 9-1(1).mp4
Saving 10-1.mp4 to 10-1.mp4
Saving 11-1.mp4 to 11-1.mp4
Saving 12-1.mp4 to 12-1.mp4
Saving 13-1.mp4 to 13-1.mp4
Saving 14-1.mp4 to 14-1.mp4
Saving 15-1.mp4 to 15-1.mp4
Saving 16-1.mp4 to 16-1.mp4
Saving 17-1.mp4 to 17-1.mp4
Saving 18-1.mp4 to 18-1.mp4
✅ 1-1.mp4 → ponerse_pie_001.mp4
✅ 2-1.mp4 → ponerse_pie_002.mp4
✅ 3-1.mp4 → ponerse_pie_003.mp4
✅ 4-1.mp4 → ponerse_pie_004.mp4
✅ 5-1.mp4 → ponerse_pie_005.mp4
✅ 6-1.mp4 → ponerse_pie_006.mp4
✅ 7-1.mp4 → ponerse_pie_007.mp4
✅ 8-1.mp4 → ponerse_pie_008.mp4
✅ 9-1(1).mp4 → ponerse_pie_009.mp4
✅ 10-1.mp4 → ponerse_pie_010.mp4
✅ 11-1.mp4 → ponerse_pie_011.mp4
✅ 12-1.mp4 → ponerse_pie_012.mp4
✅ 13-1.mp4 → ponerse_pie_013.mp4
✅ 14-1.mp4 → ponerse_pie_014.mp4
✅ 15-1.mp4 → ponerse_pie_015.mp4
✅ 16-1.mp4 → poners

## Paso 4: Configurar MediaPipe
Crear el procesador MediaPipe para extraer coordenadas de landmarks.


In [ ]:
# CONFIGURAR MEDIAPIPE
class MediaPipeProcessor:
    def __init__(self):
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(
            model_complexity=1,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        # 16 landmarks importantes
        self.landmarks = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
        self.names = [
            'L_shoulder', 'R_shoulder', 'L_elbow', 'R_elbow', 'L_wrist', 'R_wrist',
            'L_hip', 'R_hip', 'L_knee', 'R_knee', 'L_ankle', 'R_ankle',
            'L_heel', 'R_heel', 'L_foot', 'R_foot'
        ]

    def extract_landmarks(self, video_path):
        """Extraer landmarks de un video"""
        cap = cv2.VideoCapture(str(video_path))

        if not cap.isOpened():
            return None

        all_landmarks = []
        total_frames = 0
        detected_frames = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            total_frames += 1

            # Procesar con MediaPipe
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = self.pose.process(rgb)

            if results.pose_landmarks:
                detected_frames += 1
                # Extraer coordenadas
                frame_data = []
                for idx in self.landmarks:
                    lm = results.pose_landmarks.landmark[idx]
                    frame_data.extend([lm.x, lm.y, lm.z, lm.visibility])
                all_landmarks.append(frame_data)
            else:
                # Sin detección - rellenar con ceros
                all_landmarks.append([0.0] * 64)

        cap.release()

        detection_rate = (detected_frames / total_frames) * 100 if total_frames > 0 else 0

        return {
            'landmarks': np.array(all_landmarks),
            'total_frames': total_frames,
            'detected_frames': detected_frames,
            'detection_rate': detection_rate
        }

# Crear procesador
processor = MediaPipeProcessor()
print("✅ MediaPipe Processor configurado")
print(f"📊 Landmarks por frame: {len(processor.landmarks)} × 4 = 64 coordenadas")


✅ MediaPipe Processor configurado
📊 Landmarks por frame: 16 × 4 = 64 coordenadas


## Paso 5: Procesar TODOS los Videos
Procesar todos los videos subidos y generar archivos CSV con las coordenadas.


In [ ]:
# PROCESAR TODOS LOS VIDEOS
def process_all_videos():
    """Procesar todos los videos con MediaPipe"""
    print("⚙️ PROCESANDO TODOS LOS VIDEOS")
    print("=" * 40)

    all_results = []
    total_videos = 0
    successful = 0

    for activity in ACTIVITIES:
        activity_dir = Path(f"data/videos/{activity}")

        if not activity_dir.exists():
            continue

        # Encontrar videos
        videos = list(activity_dir.glob("*.mp4")) + \
                list(activity_dir.glob("*.mov")) + \
                list(activity_dir.glob("*.avi"))

        if not videos:
            print(f"⚠️ {activity}: Sin videos")
            continue

        print(f"\n🎬 {activity.replace('_', ' ').title()}: {len(videos)} videos")

        for video in videos:
            total_videos += 1

            # Procesar video
            result = processor.extract_landmarks(video)

            if result and result['detection_rate'] > 10:  # Muy permisivo
                # Crear DataFrame
                columns = []
                for name in processor.names:
                    for coord in ['x', 'y', 'z', 'visibility']:
                        columns.append(f"{name}_{coord}")

                df = pd.DataFrame(result['landmarks'], columns=columns)
                df['activity'] = activity
                df['video_file'] = video.name
                df['frame_number'] = range(len(df))

                # Guardar CSV
                csv_name = f"{video.stem}_landmarks.csv"
                csv_path = Path(f"data/landmarks/{csv_name}")
                df.to_csv(csv_path, index=False)

                successful += 1
                all_results.append({
                    'video': video.name,
                    'activity': activity,
                    'frames': result['total_frames'],
                    'detection_rate': result['detection_rate'],
                    'csv_file': csv_name
                })

                print(f"   ✅ {video.name}: {result['detection_rate']:.1f}% → {csv_name}")
            else:
                print(f"   ❌ {video.name}: Sin detección suficiente")

    # Resumen
    print(f"\n📊 RESUMEN:")
    print(f"   🎥 Videos encontrados: {total_videos}")
    print(f"   📊 CSVs generados: {successful}")
    print(f"   ✅ Éxito: {(successful/total_videos)*100:.1f}%")

    # Distribución por actividad
    activity_counts = {}
    for result in all_results:
        act = result['activity']
        activity_counts[act] = activity_counts.get(act, 0) + 1

    print(f"\n📁 DISTRIBUCIÓN:")
    for activity, count in activity_counts.items():
        print(f"   🎬 {activity.replace('_', ' ').title()}: {count} CSVs")

    # Guardar log
    with open('data/metadata/processing_log.json', 'w') as f:
        json.dump({
            'date': datetime.now().isoformat(),
            'total_videos': total_videos,
            'successful': successful,
            'results': all_results
        }, f, indent=2)

    if successful > 0:
        print(f"\n🎉 DATASET LISTO PARA EDA!")

    return all_results

print("✅ Función de procesamiento lista")


✅ Función de procesamiento lista


## Paso 6: Ejecutar Procesamiento Completo
**Ejecutar después de subir todos los videos de todas las actividades.**


In [ ]:
# EJECUTAR PROCESAMIENTO COMPLETO
print("🚀 PROCESANDO TODOS LOS VIDEOS...")

results = process_all_videos()

if results:
    print(f"\n🎉 ¡PROCESAMIENTO COMPLETADO!")
    print(f"   📊 Videos procesados: {len(results)}")
    print(f"   📁 CSVs en: data/landmarks/")
    print(f"   ✅ Listo para EDA")
else:
    print(f"\n⚠️ No se procesaron videos")
    print(f"💡 Asegúrate de subir videos en las celdas anteriores")


🚀 PROCESANDO TODOS LOS VIDEOS...
⚙️ PROCESANDO TODOS LOS VIDEOS

🎬 Caminar Hacia: 18 videos
   ✅ caminar_hacia_001.mp4: 100.0% → caminar_hacia_001_landmarks.csv
   ✅ caminar_hacia_014.mp4: 100.0% → caminar_hacia_014_landmarks.csv
   ✅ caminar_hacia_013.mp4: 100.0% → caminar_hacia_013_landmarks.csv
   ✅ caminar_hacia_015.mp4: 100.0% → caminar_hacia_015_landmarks.csv
   ✅ caminar_hacia_005.mp4: 100.0% → caminar_hacia_005_landmarks.csv
   ✅ caminar_hacia_004.mp4: 100.0% → caminar_hacia_004_landmarks.csv
   ✅ caminar_hacia_012.mp4: 100.0% → caminar_hacia_012_landmarks.csv
   ✅ caminar_hacia_010.mp4: 100.0% → caminar_hacia_010_landmarks.csv
   ✅ caminar_hacia_017.mp4: 100.0% → caminar_hacia_017_landmarks.csv
   ✅ caminar_hacia_006.mp4: 100.0% → caminar_hacia_006_landmarks.csv
   ✅ caminar_hacia_016.mp4: 100.0% → caminar_hacia_016_landmarks.csv
   ✅ caminar_hacia_007.mp4: 100.0% → caminar_hacia_007_landmarks.csv
   ✅ caminar_hacia_008.mp4: 100.0% → caminar_hacia_008_landmarks.csv
   ✅ camina

In [ ]:
# DESCARGAR TODOS LOS CSV
print("📦 Descargando archivos CSV...")
print("=" * 30)

from google.colab import files
import os

# Obtener lista de archivos CSV en la carpeta data/landmarks
csv_files = [f for f in os.listdir("data/landmarks") if f.endswith('.csv')]

if not csv_files:
    print("⚠️ No se encontraron archivos CSV en data/landmarks.")
else:
    for csv_file in csv_files:
        file_path = os.path.join("data/landmarks", csv_file)
        try:
            files.download(file_path)
            print(f"✅ Descargado: {csv_file}")
        except Exception as e:
            print(f"❌ Error al descargar {csv_file}: {e}")

print("\n🎉 Descarga completa")

📦 Descargando archivos CSV...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Descargado: girar_002_landmarks.csv


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

✅ Descargado: caminar_regreso_013_landmarks.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Descargado: girar_005_landmarks.csv

🎉 Descarga completa


In [ ]:
# ============================================
# DESCARGAR TODOS LOS CSV EN UN ZIP
# ============================================

import os
import zipfile
from pathlib import Path
from google.colab import files

print("📦 PREPARANDO DESCARGA DE TODOS LOS CSV")
print("=" * 70)

# Crear archivo ZIP
zip_filename = "landmarks_dataset_completo.zip"
landmarks_dir = Path("data/landmarks")

# Verificar que exista el directorio
if not landmarks_dir.exists():
    print("❌ Error: No existe el directorio data/landmarks")
else:
    # Obtener todos los CSV
    csv_files = sorted(landmarks_dir.glob("*.csv"))

    print(f"📁 Archivos encontrados: {len(csv_files)}")

    if len(csv_files) == 0:
        print("⚠️  No se encontraron archivos CSV")
    else:
        # Crear ZIP
        print(f"\n🔄 Comprimiendo {len(csv_files)} archivos...")

        with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for idx, csv_file in enumerate(csv_files, 1):
                # Agregar archivo al ZIP
                arcname = csv_file.name  # Nombre dentro del ZIP
                zipf.write(csv_file, arcname=arcname)

                # Mostrar progreso cada 10 archivos
                if idx % 10 == 0 or idx == len(csv_files):
                    print(f"   ✓ {idx}/{len(csv_files)} archivos agregados")

        # Tamaño del ZIP
        zip_size_mb = Path(zip_filename).stat().st_size / (1024 * 1024)

        print("\n" + "=" * 70)
        print("✅ ARCHIVO ZIP CREADO")
        print("=" * 70)
        print(f"📦 Nombre: {zip_filename}")
        print(f"📊 Archivos: {len(csv_files)}")
        print(f"💾 Tamaño: {zip_size_mb:.2f} MB")
        print("=" * 70)

        # Descargar ZIP
        print("\n🔽 Descargando archivo ZIP...")
        files.download(zip_filename)

        print("\n🎉 ¡Descarga completa!")
        print("\n💡 En tu computadora:")
        print("   1. Descomprime el archivo ZIP")
        print("   2. Encontrarás los 90 archivos CSV")


📦 PREPARANDO DESCARGA DE TODOS LOS CSV
📁 Archivos encontrados: 90

🔄 Comprimiendo 90 archivos...
   ✓ 10/90 archivos agregados
   ✓ 20/90 archivos agregados
   ✓ 30/90 archivos agregados
   ✓ 40/90 archivos agregados
   ✓ 50/90 archivos agregados
   ✓ 60/90 archivos agregados
   ✓ 70/90 archivos agregados
   ✓ 80/90 archivos agregados
   ✓ 90/90 archivos agregados

✅ ARCHIVO ZIP CREADO
📦 Nombre: landmarks_dataset_completo.zip
📊 Archivos: 90
💾 Tamaño: 3.57 MB

🔽 Descargando archivo ZIP...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🎉 ¡Descarga completa!

💡 En tu computadora:
   1. Descomprime el archivo ZIP
   2. Encontrarás los 90 archivos CSV


## Paso 7: Verificar Resultados
Verificar que todos los archivos se generaron correctamente.


In [ ]:
# VERIFICAR RESULTADOS
def show_final_stats():
    """Mostrar estadísticas finales del procesamiento"""
    print("📊 ESTADÍSTICAS FINALES")
    print("=" * 30)

    total_videos = 0
    total_csvs = 0

    for activity in ACTIVITIES:
        # Contar videos
        video_dir = Path(f"data/videos/{activity}")
        video_count = len(list(video_dir.glob("*.*"))) if video_dir.exists() else 0

        # Contar CSVs
        csv_files = list(Path("data/landmarks").glob(f"*{activity}*.csv")) if Path("data/landmarks").exists() else []
        csv_count = len(csv_files)

        total_videos += video_count
        total_csvs += csv_count

        print(f"🎬 {activity.replace('_', ' ').title()}: {video_count} videos → {csv_count} CSVs")

    print(f"\n📈 TOTAL: {total_videos} videos → {total_csvs} CSVs")

    # Verificar contenido de CSV
    if total_csvs > 0:
        csv_files = list(Path("data/landmarks").glob("*.csv"))
        sample = pd.read_csv(csv_files[0])

        print(f"\n📋 ESTRUCTURA CSV (ejemplo):")
        print(f"   Filas: {len(sample)}")
        print(f"   Columnas: {len(sample.columns)}")
        print(f"   Actividad: {sample['activity'].iloc[0] if 'activity' in sample.columns else 'N/A'}")

        print(f"\n✅ DATASET PROCESADO CORRECTAMENTE")
        print(f"🎯 Continuar con: 02_eda_inicial.ipynb")

        return True
    else:
        print(f"\n❌ No se generaron CSVs")
        return False

# Verificar
final_verification = show_final_stats()


📊 ESTADÍSTICAS FINALES
🎬 Caminar Hacia: 18 videos → 18 CSVs
🎬 Caminar Regreso: 18 videos → 18 CSVs
🎬 Girar: 18 videos → 18 CSVs
🎬 Sentarse: 18 videos → 18 CSVs
🎬 Ponerse Pie: 18 videos → 18 CSVs

📈 TOTAL: 90 videos → 90 CSVs

📋 ESTRUCTURA CSV (ejemplo):
   Filas: 70
   Columnas: 67
   Actividad: girar

✅ DATASET PROCESADO CORRECTAMENTE
🎯 Continuar con: 02_eda_inicial.ipynb
